DATABASE connection


In [1]:
import yaml
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd

In [2]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [3]:
config = load_config()
db_config = config["database"]

# Load credentials
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# DB connection
conn = psycopg2.connect(
    dbname="postgres", user=db_user, password=db_password, host=db_host, port=db_port
)
conn.autocommit = True

In [4]:
db_name = "etl_db"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close()

Base de datos 'etl_db' creada exitosamente.


In [5]:
engine = create_engine(
    f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)

In [6]:
types = pd.api.types

def map_dtype(dtype):
    dtype_mapping = {
        types.is_integer_dtype: "INTEGER",
        types.is_float_dtype: "REAL",
        types.is_bool_dtype: "BOOLEAN",
        types.is_datetime64_any_dtype: "DATETIME",
    }

    return next(
        (sql_type for check, sql_type in dtype_mapping.items() if check(dtype)),
        "TEXT",
    )

In [7]:
table_name_tabla_etl = "tabla_etl"

df = pd.read_csv("./dataset/candidates.csv", sep=";")

formatted_columns = [col.lower().replace(' ', '_') for col in df.columns]

tabla_etl_columns_def = ", ".join(f"{col} {map_dtype(dtype)}" for col, dtype in zip(formatted_columns, df.dtypes))

sql_create_tabla_etl = f"CREATE TABLE IF NOT EXISTS {table_name_tabla_etl} ({tabla_etl_columns_def})"

with engine.connect() as conn:
    conn.execute(text(sql_create_tabla_etl))
    conn.commit()
    print(f"Tabla {table_name_tabla_etl} creada exitosamente en PostgreSQL.")

Tabla tabla_etl creada exitosamente en PostgreSQL.


In [8]:
sql_insert = f"INSERT INTO {table_name_tabla_etl} ({', '.join(formatted_columns)}) VALUES ({', '.join([f':param{i}' for i in range(len(df.columns))])})"
print(sql_insert)

values = [
    dict(zip([f"param{i}" for i in range(len(df.columns))], row))
    for row in df.to_numpy()
]

with engine.connect() as conn:
    conn.execute(text(sql_insert), values)
    conn.commit()
    print("Datos insertados exitosamente en 'tabla_etl'.")

INSERT INTO tabla_etl (first_name, last_name, email, application_date, country, yoe, seniority, technology, code_challenge_score, technical_interview_score) VALUES (:param0, :param1, :param2, :param3, :param4, :param5, :param6, :param7, :param8, :param9)
Datos insertados exitosamente en 'tabla_etl'.


In [9]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM tabla_etl;"))
    rows = result.fetchall()

In [10]:
with engine.connect() as conn:
    df = pd.read_sql("SELECT * FROM tabla_etl", conn)

In [11]:
df.head(10)

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
5,Alec,Abbott,juanita_hansen@gmail.com,2019-08-17,Zimbabwe,8,Junior,Adobe Experience Manager,2,9
6,Allison,Jacobs,alba_rolfson27@yahoo.com,2018-05-18,Wallis and Futuna,19,Trainee,Sales,2,9
7,Nya,Skiles,madisen.zulauf@gmail.com,2021-12-09,Myanmar,1,Lead,Mulesoft,2,5
8,Mose,Lakin,dale_murazik@hotmail.com,2018-03-13,Italy,18,Lead,Social Media Community Management,7,10
9,Terrance,Zieme,dustin31@hotmail.com,2022-04-08,Timor-Leste,25,Lead,DevOps,2,0


Transform phase

In [12]:
df_transformed = df.copy()
df_transformed["hired"] = (df_transformed["code_challenge_score"] >= 7) & (df_transformed["technical_interview_score"] >= 7)

df_transformed.head(10)

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,hired
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,False
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,False
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,True
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,False
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,True
5,Alec,Abbott,juanita_hansen@gmail.com,2019-08-17,Zimbabwe,8,Junior,Adobe Experience Manager,2,9,False
6,Allison,Jacobs,alba_rolfson27@yahoo.com,2018-05-18,Wallis and Futuna,19,Trainee,Sales,2,9,False
7,Nya,Skiles,madisen.zulauf@gmail.com,2021-12-09,Myanmar,1,Lead,Mulesoft,2,5,False
8,Mose,Lakin,dale_murazik@hotmail.com,2018-03-13,Italy,18,Lead,Social Media Community Management,7,10,True
9,Terrance,Zieme,dustin31@hotmail.com,2022-04-08,Timor-Leste,25,Lead,DevOps,2,0,False


In [13]:
table_name_transformed_data = 'transformed_data_etl'

with engine.connect() as conn:
    df_transformed.to_sql(table_name_transformed_data, con=engine, if_exists="append", index=False)

print(f"Transformed data stored successfully in '{table_name_transformed_data}'.")

Transformed data stored successfully in 'transformed_data_etl'.


In [14]:
with engine.connect() as conn:
    db_transformed_df = pd.read_sql(text(f"SELECT * FROM {table_name_transformed_data}"), conn)
    
db_transformed_df.head(10)

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,hired
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,False
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,False
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,True
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,False
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,True
5,Alec,Abbott,juanita_hansen@gmail.com,2019-08-17,Zimbabwe,8,Junior,Adobe Experience Manager,2,9,False
6,Allison,Jacobs,alba_rolfson27@yahoo.com,2018-05-18,Wallis and Futuna,19,Trainee,Sales,2,9,False
7,Nya,Skiles,madisen.zulauf@gmail.com,2021-12-09,Myanmar,1,Lead,Mulesoft,2,5,False
8,Mose,Lakin,dale_murazik@hotmail.com,2018-03-13,Italy,18,Lead,Social Media Community Management,7,10,True
9,Terrance,Zieme,dustin31@hotmail.com,2022-04-08,Timor-Leste,25,Lead,DevOps,2,0,False
